In [ ]:
%%writefile rt_simulation.c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <sys/stat.h>

#define Nx 64
#define Ny 128
#define Lx 1.0
#define Ly 2.0
#define dx (Lx / Nx)
#define dy (Ly / Ny)
#define gamma 1.4
#define ga -10.0
#define C 0.2
#define save_interval 10

void update_diffusion_coeffs(double dt, double *k1, double *k2, double *k3) {
    *k1 = 0.0125 * (dx * dx) / (2 * dt);
    *k2 = 0.125 * (dx * dx) / (2 * dt);
    *k3 = 0.0125 * (dx * dx) / (2 * dt);
}

void initialize_grid(double r[Nx][Ny], double ru[Nx][Ny], double rv[Nx][Ny],
                    double e[Nx][Ny], double p[Nx][Ny]) {
    int i, j;
    double y, u[Nx][Ny], v[Nx][Ny];

    for (i = 0; i < Nx; i++) {
        for (j = 0; j < Ny; j++) {
            y = (j + 0.5) * dy;
            if (y >= Ly / 2) {
                r[i][j] = 2.0;
            } else {
                r[i][j] = 1.0;
            }
            u[i][j] = 0.0;
            v[i][j] = 0.0;
        }
    }

    for (i = 0; i < Nx; i++) {
        for (j = 0; j < Ny; j++) {
            y = (j + 0.5) * dy;
            if (fabs(y - Ly / 2) <= 0.05) {
                v[i][j] = 1e-3 * (2 * ((double)rand() / RAND_MAX) - 1);
            }
        }
    }

    for (i = 0; i < Nx; i++) {
        for (j = 0; j < Ny; j++) {
            y = (j + 0.5) * dy;
            p[i][j] = 40.0 + r[i][j] * ga * (y - Ly / 2);
            ru[i][j] = r[i][j] * u[i][j];
            rv[i][j] = r[i][j] * v[i][j];
            e[i][j] = p[i][j] / (gamma - 1) + 0.5 * r[i][j] * (u[i][j] * u[i][j] + v[i][j] * v[i][j]);
        }
    }
}

double compute_dt(double r[Nx][Ny], double ru[Nx][Ny], double rv[Nx][Ny], double p[Nx][Ny]) {
    double u[Nx][Ny], v[Nx][Ny], c[Nx][Ny];
    double max_speed = 0.0;
    int i, j;

    for (i = 0; i < Nx; i++) {
        for (j = 0; j < Ny; j++) {
            if (r[i][j] > 1e-10) {
                u[i][j] = ru[i][j] / r[i][j];
                v[i][j] = rv[i][j] / r[i][j];
            } else {
                u[i][j] = 0.0;
                v[i][j] = 0.0;
            }
            c[i][j] = sqrt(gamma * p[i][j] / r[i][j]);
            double speed = fmax(fmax(fabs(u[i][j]), fabs(v[i][j])), c[i][j]);
            if (speed > max_speed) {
                max_speed = speed;
            }
        }
    }

    return C * fmin(dx, dy) / max_speed;
}

void compute_derivatives(double f[Nx][Ny], double df_dx[Nx][Ny], double df_dy[Nx][Ny],
                        double d2f_dx2[Nx][Ny], double d2f_dy2[Nx][Ny]) {
    int i, j;
    for (i = 1; i < Nx - 1; i++) {
        for (j = 1; j < Ny - 1; j++) {
            df_dx[i][j] = (f[i + 1][j] - f[i - 1][j]) / (2 * dx);
            df_dy[i][j] = (f[i][j + 1] - f[i][j - 1]) / (2 * dy);
            d2f_dx2[i][j] = (f[i + 1][j] - 2 * f[i][j] + f[i - 1][j]) / (dx * dx);
            d2f_dy2[i][j] = (f[i][j + 1] - 2 * f[i][j] + f[i][j - 1]) / (dy * dy);
        }
    }
}

void compute_rhs(double r[Nx][Ny], double ru[Nx][Ny], double rv[Nx][Ny], double e[Nx][Ny],
                 double p[Nx][Ny], double k1, double k2, double k3,
                 double rhs_r[Nx][Ny], double rhs_ru[Nx][Ny], double rhs_rv[Nx][Ny], double rhs_e[Nx][Ny]) {
    int i, j;
    double u[Nx][Ny], v[Nx][Ny];
    double ruu[Nx][Ny], ruv[Nx][Ny], rvv[Nx][Ny], eu_p[Nx][Ny], ev_p[Nx][Ny];
    double dr_dx[Nx][Ny], dr_dy[Nx][Ny], d2r_dx2[Nx][Ny], d2r_dy2[Nx][Ny];
    double dru_dx[Nx][Ny], dru_dy[Nx][Ny], d2ru_dx2[Nx][Ny], d2ru_dy2[Nx][Ny];
    double drv_dx[Nx][Ny], drv_dy[Nx][Ny], d2rv_dx2[Nx][Ny], d2rv_dy2[Nx][Ny];
    double de_dx[Nx][Ny], de_dy[Nx][Ny], d2e_dx2[Nx][Ny], d2e_dy2[Nx][Ny];
    double dp_dx[Nx][Ny], dp_dy[Nx][Ny], dummy1[Nx][Ny], dummy2[Nx][Ny];
    double druu_dx[Nx][Ny], druv_dx[Nx][Ny], druv_dy[Nx][Ny], drvv_dy[Nx][Ny], deup_dx[Nx][Ny], devp_dy[Nx][Ny];

    for (i = 0; i < Nx; i++) {
        for (j = 0; j < Ny; j++) {
            if (r[i][j] > 1e-10) {
                u[i][j] = ru[i][j] / r[i][j];
                v[i][j] = rv[i][j] / r[i][j];
            } else {
                u[i][j] = 0.0;
                v[i][j] = 0.0;
            }
            ruu[i][j] = ru[i][j] * u[i][j];
            ruv[i][j] = ru[i][j] * v[i][j];
            rvv[i][j] = rv[i][j] * v[i][j];
            eu_p[i][j] = u[i][j] * (e[i][j] + p[i][j]);
            ev_p[i][j] = v[i][j] * (e[i][j] + p[i][j]);
        }
    }

    compute_derivatives(r, dr_dx, dr_dy, d2r_dx2, d2r_dy2);
    compute_derivatives(ru, dru_dx, dru_dy, d2ru_dx2, d2ru_dy2);
    compute_derivatives(rv, drv_dx, drv_dy, d2rv_dx2, d2rv_dy2);
    compute_derivatives(e, de_dx, de_dy, d2e_dx2, d2e_dy2);
    compute_derivatives(p, dp_dx, dp_dy, dummy1, dummy2);
    compute_derivatives(ruu, druu_dx, dummy1, dummy2, dummy2);
    compute_derivatives(ruv, druv_dx, druv_dy, dummy1, dummy2);
    compute_derivatives(rvv, dummy1, drvv_dy, dummy2, dummy2);
    compute_derivatives(eu_p, deup_dx, dummy1, dummy2, dummy2);
    compute_derivatives(ev_p, dummy1, devp_dy, dummy2, dummy2);

    for (i = 1; i < Nx - 1; i++) {
        for (j = 1; j < Ny - 1; j++) {
            rhs_r[i][j] = -(dru_dx[i][j] + drv_dy[i][j]) + k1 * (d2r_dx2[i][j] + d2r_dy2[i][j]);
            rhs_ru[i][j] = -(druu_dx[i][j] + druv_dy[i][j]) - dp_dx[i][j] + k2 * d2ru_dx2[i][j];
            rhs_rv[i][j] = -(druv_dx[i][j] + drvv_dy[i][j]) - dp_dy[i][j] + ga * r[i][j] + k2 * d2rv_dy2[i][j];
            rhs_e[i][j] = -(deup_dx[i][j] + devp_dy[i][j]) + ga * r[i][j] * v[i][j] + k3 * (d2e_dx2[i][j] + d2e_dy2[i][j]);
        }
    }
}

void apply_boundary_conditions(double r[Nx][Ny], double ru[Nx][Ny], double rv[Nx][Ny],
                              double e[Nx][Ny], double p[Nx][Ny]) {
    int i;
    for (i = 0; i < Ny; i++) {
        r[0][i] = r[Nx - 2][i];
        r[Nx - 1][i] = r[1][i];
        ru[0][i] = ru[Nx - 2][i];
        ru[Nx - 1][i] = ru[1][i];
        rv[0][i] = rv[Nx - 2][i];
        rv[Nx - 1][i] = rv[1][i];
        e[0][i] = e[Nx - 2][i];
        e[Nx - 1][i] = e[1][i];
        p[0][i] = p[Nx - 2][i];
        p[Nx - 1][i] = p[1][i];
    }

    for (i = 0; i < Nx; i++) {
        rv[i][0] = 0;
        rv[i][Ny - 1] = 0;
        r[i][0] = r[i][1];
        ru[i][0] = ru[i][1];
        p[i][0] = p[i][1];
        e[i][0] = p[i][0] / (gamma - 1) + 0.5 * (ru[i][0] * ru[i][0] + rv[i][0] * rv[i][0]) / r[i][0];
        r[i][Ny - 1] = r[i][Ny - 2];
        ru[i][Ny - 1] = ru[i][Ny - 2];
        p[i][Ny - 1] = p[i][Ny - 2];
        e[i][Ny - 1] = p[i][Ny - 1] / (gamma - 1) + 0.5 * (ru[i][Ny - 1] * ru[i][Ny - 1] + rv[i][Ny - 1] * rv[i][Ny - 1]) / r[i][Ny - 1];
    }
}

void update_pressure(double r[Nx][Ny], double ru[Nx][Ny], double rv[Nx][Ny],
                    double e[Nx][Ny], double p[Nx][Ny]) {
    int i, j;
    double u, v;
    for (i = 0; i < Nx; i++) {
        for (j = 0; j < Ny; j++) {
            if (r[i][j] > 1e-10) {
                u = ru[i][j] / r[i][j];
                v = rv[i][j] / r[i][j];
            } else {
                u = 0.0;
                v = 0.0;
            }
            p[i][j] = (gamma - 1) * (e[i][j] - 0.5 * r[i][j] * (u * u + v * v));
            if (p[i][j] < 1e-10) {
                p[i][j] = 1e-10;
            }
        }
    }
}

void save_density(double r[Nx][Ny], int step) {
    char filename[100];
    sprintf(filename, "density/density_step_%d.txt", step);
    FILE *fp = fopen(filename, "w");
    if (fp == NULL) {
        printf("Error opening file %s\n", filename);
        return;
    }
    for (int i = 0; i < Nx; i++) {
        for (int j = 0; j < Ny; j++) {
            fprintf(fp, "%e ", r[i][j]);
        }
        fprintf(fp, "\n");
    }
    fclose(fp);
}

void euler_step(double r[Nx][Ny], double ru[Nx][Ny], double rv[Nx][Ny], double e[Nx][Ny],
                double p[Nx][Ny], double dt) {
    double k1, k2, k3;
    double rhs_r[Nx][Ny], rhs_ru[Nx][Ny], rhs_rv[Nx][Ny], rhs_e[Nx][Ny];
    double r_new[Nx][Ny], ru_new[Nx][Ny], rv_new[Nx][Ny], e_new[Nx][Ny], p_new[Nx][Ny];
    int i, j;

    update_diffusion_coeffs(dt, &k1, &k2, &k3);
    compute_rhs(r, ru, rv, e, p, k1, k2, k3, rhs_r, rhs_ru, rhs_rv, rhs_e);

    for (i = 0; i < Nx; i++) {
        for (j = 0; j < Ny; j++) {
            r_new[i][j] = r[i][j] + dt * rhs_r[i][j];
            ru_new[i][j] = ru[i][j] + dt * rhs_ru[i][j];
            rv_new[i][j] = rv[i][j] + dt * rhs_rv[i][j];
            e_new[i][j] = e[i][j] + dt * rhs_e[i][j];
        }
    }

    update_pressure(r_new, ru_new, rv_new, e_new, p_new);
    apply_boundary_conditions(r_new, ru_new, rv_new, e_new, p_new);

    for (i = 0; i < Nx; i++) {
        for (j = 0; j < Ny; j++) {
            r[i][j] = r_new[i][j];
            ru[i][j] = ru_new[i][j];
            rv[i][j] = rv_new[i][j];
            e[i][j] = e_new[i][j];
            p[i][j] = p_new[i][j];
        }
    }
}

void rk2_step(double r[Nx][Ny], double ru[Nx][Ny], double rv[Nx][Ny], double e[Nx][Ny],
              double p[Nx][Ny], double dt) {
    double k1, k2, k3;
    double rhs_r[Nx][Ny], rhs_ru[Nx][Ny], rhs_rv[Nx][Ny], rhs_e[Nx][Ny];
    double r_star[Nx][Ny], ru_star[Nx][Ny], rv_star[Nx][Ny], e_star[Nx][Ny], p_star[Nx][Ny];
    double rhs_r_star[Nx][Ny], rhs_ru_star[Nx][Ny], rhs_rv_star[Nx][Ny], rhs_e_star[Nx][Ny];
    double r_new[Nx][Ny], ru_new[Nx][Ny], rv_new[Nx][Ny], e_new[Nx][Ny], p_new[Nx][Ny];
    int i, j;

    update_diffusion_coeffs(dt, &k1, &k2, &k3);
    compute_rhs(r, ru, rv, e, p, k1, k2, k3, rhs_r, rhs_ru, rhs_rv, rhs_e);

    for (i = 0; i < Nx; i++) {
        for (j = 0; j < Ny; j++) {
            r_star[i][j] = r[i][j] + dt * rhs_r[i][j];
            ru_star[i][j] = ru[i][j] + dt * rhs_ru[i][j];
            rv_star[i][j] = rv[i][j] + dt * rhs_rv[i][j];
            e_star[i][j] = e[i][j] + dt * rhs_e[i][j];
        }
    }

    update_pressure(r_star, ru_star, rv_star, e_star, p_star);
    apply_boundary_conditions(r_star, ru_star, rv_star, e_star, p_star);

    compute_rhs(r_star, ru_star, rv_star, e_star, p_star, k1, k2, k3, rhs_r_star, rhs_ru_star, rhs_rv_star, rhs_e_star);

    for (i = 0; i < Nx; i++) {
        for (j = 0; j < Ny; j++) {
            r_new[i][j] = 0.5 * r[i][j] + 0.5 * (r_star[i][j] + dt * rhs_r_star[i][j]);
            ru_new[i][j] = 0.5 * ru[i][j] + 0.5 * (ru_star[i][j] + dt * rhs_ru_star[i][j]);
            rv_new[i][j] = 0.5 * rv[i][j] + 0.5 * (rv_star[i][j] + dt * rhs_rv_star[i][j]);
            e_new[i][j] = 0.5 * e[i][j] + 0.5 * (e_star[i][j] + dt * rhs_e_star[i][j]);
        }
    }

    update_pressure(r_new, ru_new, rv_new, e_new, p_new);
    apply_boundary_conditions(r_new, ru_new, rv_new, e_new, p_new);

    for (i = 0; i < Nx; i++) {
        for (j = 0; j < Ny; j++) {
            r[i][j] = r_new[i][j];
            ru[i][j] = ru_new[i][j];
            rv[i][j] = rv_new[i][j];
            e[i][j] = e_new[i][j];
            p[i][j] = p_new[i][j];
        }
    }
}

int main() {
    double r[Nx][Ny], ru[Nx][Ny], rv[Nx][Ny], e[Nx][Ny], p[Nx][Ny];
    double t = 0.0, max_time = 15, dt;
    int step = 0;
    int use_rk2 = 1;

    system("rm -rf density"); // Remove existing density directory and its contents
    mkdir("density", 0755);
    initialize_grid(r, ru, rv, e, p);
    save_density(r, 0);

    while (t < max_time) {
        dt = compute_dt(r, ru, rv, p);
        if (t + dt > max_time) {
            dt = max_time - t;
        }

        if (use_rk2) {
            rk2_step(r, ru, rv, e, p, dt);
        } else {
            euler_step(r, ru, rv, e, p, dt);
        }

        t += dt;
        step++;

        if (step % save_interval == 0) {
            printf("Step %d: t = %.4f, dt = %.4f\n", step, t, dt);
            save_density(r, step);
        }
    }

    save_density(r, step);
    return 0;
}


Writing rt_simulation.c


In [ ]:
!gcc rt_simulation.c -o rt_simulation -lm


In [ ]:
!./rt_simulation


Step 10: t = 0.0037, dt = 0.0004
Step 20: t = 0.0075, dt = 0.0004
Step 30: t = 0.0112, dt = 0.0004
Step 40: t = 0.0150, dt = 0.0004
Step 50: t = 0.0187, dt = 0.0004
Step 60: t = 0.0225, dt = 0.0004
Step 70: t = 0.0262, dt = 0.0004
Step 80: t = 0.0300, dt = 0.0004
Step 90: t = 0.0337, dt = 0.0004
Step 100: t = 0.0374, dt = 0.0004
Step 110: t = 0.0412, dt = 0.0004
Step 120: t = 0.0449, dt = 0.0004
Step 130: t = 0.0487, dt = 0.0004
Step 140: t = 0.0524, dt = 0.0004
Step 150: t = 0.0562, dt = 0.0004
Step 160: t = 0.0599, dt = 0.0004
Step 170: t = 0.0637, dt = 0.0004
Step 180: t = 0.0674, dt = 0.0004
Step 190: t = 0.0712, dt = 0.0004
Step 200: t = 0.0749, dt = 0.0004
Step 210: t = 0.0787, dt = 0.0004
Step 220: t = 0.0824, dt = 0.0004
Step 230: t = 0.0862, dt = 0.0004
Step 240: t = 0.0899, dt = 0.0004
Step 250: t = 0.0936, dt = 0.0004
Step 260: t = 0.0974, dt = 0.0004
Step 270: t = 0.1011, dt = 0.0004
Step 280: t = 0.1049, dt = 0.0004
Step 290: t = 0.1086, dt = 0.0004
Step 300: t = 0.1124, d

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

Nx = 64
Ny = 128
Lx = 1.0
Ly = 2.0
max_time = 15.0
save_interval = 10

density_dir = "density"

density_files = [f for f in os.listdir(density_dir) if f.startswith("density_step_") and f.endswith(".txt")]
steps = [int(f.split("_")[2].split(".")[0]) for f in density_files]
files_and_steps = sorted(zip(density_files, steps), key=lambda x: x[1])
filenames = [f[0] for f in files_and_steps]
steps = [f[1] for f in files_and_steps]

fig, ax = plt.subplots(figsize=(8, 8))
x = np.linspace(0, Lx, Nx)
y = np.linspace(0, Ly, Ny)
X, Y = np.meshgrid(x, y)

initial_data = np.loadtxt(os.path.join(density_dir, filenames[0])).reshape(Nx, Ny)
im = ax.pcolormesh(X, Y, initial_data.T, cmap='viridis')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_aspect('equal')
ax.set_title(f"Step {steps[0]}, t ≈ 0.000")
fig.colorbar(im, ax=ax, label='Density')

def update(frame):
    filepath = os.path.join(density_dir, filenames[frame])
    density = np.loadtxt(filepath).reshape(Nx, Ny)
    im.set_array(density.T.ravel())
    time = steps[frame] * max_time / max(steps)
    ax.set_title(f"Step {steps[frame]}, t ≈ {time:.3f}")
    return [im]

ani = FuncAnimation(fig, update, frames=len(filenames), interval=100, blit=True)

ani.save('density_animation.mp4', writer='ffmpeg', fps=10)

plt.close(fig)
# HTML(ani.to_html5_video())